# 01 — Data Collection Notebook
Federal Funds Rate Prediction Using Big Mac Index & FRED Data

**Goal:** Collect and align raw macroeconomic data needed for feature engineering and modeling.

**Sources:**
- Federal Reserve Economic Data (FRED API)
- The Economist Big Mac Index (CSV)

In [142]:
import os
import pandas as pd
import numpy as np
from fredapi import Fred
from dotenv import load_dotenv

pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", None)

print("Libraries loaded.")

Libraries loaded.


In [143]:
# Load .env file
load_dotenv()

fred_api_key = os.getenv("FRED_API_KEY")
if fred_api_key is None:
    raise ValueError("FRED_API_KEY not found. Check your .env file.")

fred = Fred(api_key=fred_api_key)
print("FRED API connected.")

FRED API connected.


In [144]:
bigmac_path = "data/raw/big-mac-source-data.csv"
bigmac = pd.read_csv(bigmac_path)
print("Big Mac Index loaded. Rows:", len(bigmac))

bigmac_us = bigmac[bigmac['iso_a3'] == 'USA'].copy()
bigmac_us = bigmac_us[['date', 'local_price']]
bigmac_us['date'] = pd.to_datetime(bigmac_us['date'])
bigmac_us = bigmac_us.set_index('date').resample("ME").ffill()
bigmac_us.head(100)

Big Mac Index loaded. Rows: 2302


,local_price
date,
2000-04-30,2.24
2000-05-31,2.24
2000-06-30,2.24
2000-07-31,2.24
2000-08-31,2.24
...,...
2008-03-31,3.00
2008-04-30,3.00
2008-05-31,3.00


In [145]:
# Pre-2008 single target rate
dfedtar = fred.get_series("DFEDTAR")  # single target rate (discontinued)
dfedtar = dfedtar.to_frame("fed_funds_target_single")
dfedtar.index = pd.to_datetime(dfedtar.index)

# Convert single target into pseudo upper/lower range
dfedtar["fed_funds_lower"] = dfedtar["fed_funds_target_single"]
dfedtar["fed_funds_upper"] = dfedtar["fed_funds_target_single"]
dfedtar["fed_funds_mid"]   = dfedtar["fed_funds_target_single"]

# Keep only columns we'll use
dfedtar = dfedtar[["fed_funds_lower", "fed_funds_upper", "fed_funds_mid"]]

# Post-2008 target range 
ffr_upper = fred.get_series("DFEDTARU").to_frame("fed_funds_upper")
ffr_lower = fred.get_series("DFEDTARL").to_frame("fed_funds_lower")

ffr_upper.index = pd.to_datetime(ffr_upper.index)
ffr_lower.index = pd.to_datetime(ffr_lower.index)

# Merge modern upper/lower
ffr_post = pd.concat([ffr_lower, ffr_upper], axis=1)

# Compute midpoint
ffr_post["fed_funds_mid"] = (ffr_post["fed_funds_lower"] + ffr_post["fed_funds_upper"]) / 2

# Stitch pre + post together
ffr_pre = dfedtar[dfedtar.index < "2008-12-01"]
ffr_full = pd.concat([ffr_pre, ffr_post], axis=0)
ffr_full = ffr_full.sort_index()

# Convert to monthly frequency (M) and forward fill
ffr_full = ffr_full.resample("M").ffill()

ffr_full.head(20)


C:\Users\schmi\AppData\Local\Temp\ipykernel_34924\1230817107.py:33: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  ffr_full = ffr_full.resample("M").ffill()


,fed_funds_lower,fed_funds_upper,fed_funds_mid
1982-09-30,10.2500,10.2500,10.2500
1982-10-31,9.5000,9.5000,9.5000
1982-11-30,9.0000,9.0000,9.0000
1982-12-31,8.5000,8.5000,8.5000
1983-01-31,8.5000,8.5000,8.5000
1983-02-28,8.5000,8.5000,8.5000
1983-03-31,8.6250,8.6250,8.6250
1983-04-30,8.6250,8.6250,8.6250
1983-05-31,8.7500,8.7500,8.7500
1983-06-30,9.0000,9.0000,9.0000


In [146]:
series_dict = {
    "CPIAUCSL": "cpi",
    "UNRATE": "unemployment_rate",
    "M2SL": "m2_money_supply",
    "DGS10": "treasury_10yr_yield",
    "T10Y2Y": "yield_curve_spread"
}

fred_frames = []

for series_id, colname in series_dict.items():
    data = fred.get_series(series_id)
    df = data.to_frame(name=colname)
    df.index = pd.to_datetime(df.index)
    df = df.resample("ME").mean()
    fred_frames.append(df)

fred_combined = pd.concat(fred_frames, axis=1)
fred_combined.head()

,cpi,unemployment_rate,m2_money_supply,treasury_10yr_yield,yield_curve_spread
1947-01-31,21.48,NaN,NaN,NaN,NaN
1947-02-28,21.62,NaN,NaN,NaN,NaN
1947-03-31,22.00,NaN,NaN,NaN,NaN
1947-04-30,22.00,NaN,NaN,NaN,NaN
1947-05-31,21.95,NaN,NaN,NaN,NaN


In [147]:
#Build FRED-only dataset (no Big Mac involved)

# Step 1: Determine earliest usable date across all FRED series
fred_start = fred_combined.dropna().index.min()
for col in fred_combined.columns:
    print(col, "starts at", fred_combined[col].dropna().index.min())

# Step 2: Build full monthly index from earliest date → latest date
full_index_fred = pd.date_range(
    start=fred_start,
    end=max(ffr_full.index.max(), fred_combined.index.max()),
    freq="ME"
)

# Step 3: Reindex both datasets to this timeline
ffr_fred = ffr_full.reindex(full_index_fred)
macro_fred = fred_combined.reindex(full_index_fred)

# Step 4: Forward-fill (Fed + macro indicators)
ffr_fred = ffr_fred.ffill()
macro_fred = macro_fred.ffill()

# Step 5: Combine into a single dataset
fred_only = pd.concat([ffr_fred, macro_fred], axis=1)

# Optional: Drop rows if any leading NaNs slipped in
fred_only = fred_only.dropna()

fred_only.tail(20)


cpi starts at 1947-01-31 00:00:00
unemployment_rate starts at 1948-01-31 00:00:00
m2_money_supply starts at 1959-01-31 00:00:00
treasury_10yr_yield starts at 1962-01-31 00:00:00
yield_curve_spread starts at 1976-06-30 00:00:00


,fed_funds_lower,fed_funds_upper,fed_funds_mid,cpi,unemployment_rate,m2_money_supply,treasury_10yr_yield,yield_curve_spread
2024-04-30,5.25,5.50,5.375,313.016,3.9,20928.9,4.539091,-0.334091
2024-05-31,5.25,5.50,5.375,313.140,4.0,20989.4,4.482273,-0.374091
2024-06-30,5.25,5.50,5.375,313.131,4.1,21053.0,4.305263,-0.431579
2024-07-31,5.25,5.50,5.375,313.566,4.2,21084.2,4.248636,-0.247273
2024-08-31,5.25,5.50,5.375,314.131,4.2,21171.0,3.870909,-0.094545
2024-09-30,4.75,5.00,4.875,314.851,4.1,21257.5,3.723500,0.101000
2024-10-31,4.75,5.00,4.875,315.564,4.1,21308.1,4.095455,0.123182
2024-11-30,4.50,4.75,4.625,316.449,4.2,21407.9,4.355789,0.098421
2024-12-31,4.25,4.50,4.375,317.603,4.1,21424.5,4.391429,0.165714
2025-01-31,4.25,4.50,4.375,319.086,4.0,21492.4,4.629048,0.357143


In [148]:
# Align all datasets to Big Mac Index availability

# 1. Big Mac determines the earliest valid date
bigmac_start = bigmac_us.dropna().index.min()

# 2. Full monthly index from Big Mac start → most recent data available
full_index = pd.date_range(
    start=bigmac_start,
    end=bigmac_us.index.max(),
    freq="ME"
)

# 3. Reindex all datasets to this unified timeline
ffr = ffr_full.reindex(full_index)
bigmac_us = bigmac_us.reindex(full_index)
fred_combined = fred_combined.reindex(full_index)

# 4. Forward-fill everything except Big Mac (already correct frequency)
ffr = ffr.ffill()
fred_combined = fred_combined.ffill()

# 5. Final unified dataset
combined = pd.concat([ffr, bigmac_us, fred_combined], axis=1)
combined = combined.sort_index()
combined.tail(20)


,fed_funds_lower,fed_funds_upper,fed_funds_mid,local_price,cpi,unemployment_rate,m2_money_supply,treasury_10yr_yield,yield_curve_spread
2023-06-30,5.00,5.25,5.125,5.36,304.099,3.6,20782.7,3.748095,-0.891429
2023-07-31,5.25,5.50,5.375,5.58,304.615,3.5,20772.0,3.899500,-0.929000
2023-08-31,5.25,5.50,5.375,5.58,306.138,3.7,20746.9,4.167826,-0.733913
2023-09-30,5.25,5.50,5.375,5.58,307.374,3.8,20720.4,4.379000,-0.643000
2023-10-31,5.25,5.50,5.375,5.58,307.653,3.9,20687.3,4.798095,-0.269524
2023-11-30,5.25,5.50,5.375,5.58,308.087,3.7,20696.7,4.502857,-0.379524
2023-12-31,5.25,5.50,5.375,5.58,308.735,3.8,20701.3,4.020000,-0.437000
2024-01-31,5.25,5.50,5.375,5.69,309.794,3.7,20773.1,4.058095,-0.264762
2024-02-29,5.25,5.50,5.375,5.69,311.022,3.9,20861.1,4.207500,-0.336500
2024-03-31,5.25,5.50,5.375,5.69,312.107,3.9,20900.9,4.208500,-0.379500


In [149]:
missing_report = combined.isna().sum()
missing_report

# missing_report = fred_only.isna().sum()
# missing_report

fed_funds_lower        0
fed_funds_upper        0
fed_funds_mid          0
local_price            0
cpi                    0
unemployment_rate      0
m2_money_supply        0
treasury_10yr_yield    0
yield_curve_spread     0
dtype: int64

In [150]:
combined_output_path = "data/raw/combined_raw.csv"
combined.to_csv(combined_output_path)
print("Raw dataset saved to:", combined_output_path)

output_path = "data/raw/fred_only.csv"
fred_only.to_csv(output_path)
print("Raw dataset saved to:", output_path)


Raw dataset saved to: data/raw/combined_raw.csv
Raw dataset saved to: data/raw/fred_only.csv
